In [ ]:
!pip install dicom2nifti



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 43 kB 938 kB/s 
     |████████████████████████████████| 2.0 MB 9.0 MB/s 
     |████████████████████████████████| 12.8 MB 22.3 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 990 kB 5.1 MB/s 


In [ ]:
import os
import numpy as np
from glob import glob
import shutil
import dicom2nifti
from tqdm import tqdm
import nibabel as nib


In [ ]:

#Function to create groups of 65 dcom files to divide every paitent with unique no.
def make_group(in_path,out_path):
    for patient in glob(in_path + '/*'):
            patient_name = os.path.basename(os.path.normpath(patient))

            number_folders = int(len(glob(patient + '/*')) / 65)

            for i in range(number_folders):
                out_path_name = os.path.join(out_path, patient_name + '_' + str(i))
                os.mkdir(out_path_name)

                
                for i, file in enumerate(glob(patient + '/*')):
                    if i == Number_slices + 1:
                        break

                    shutil.move(file, out_path_name)

In [ ]:
#Grouping labels into bundle of 65.

in_path = '/content/drive/MyDrive/Dataset/decom_files/labels'
out_path = '/content/drive/MyDrive/Dataset/Data_Train_Test/TrainSegmentation'

make_group(in_path, out_path)

#grouping images into bundle of 65.

in_path_images = '/content/drive/MyDrive/Dataset/decom_files/images'
out_path_images = '/content/drive/MyDrive/Dataset/Data_Train_Test/TrainVolumes'

make_group(in_path_images,out_path_images)

In [ ]:
#Converting groups of decoms files into Nifti files. 
def dicom_to_nifti(in_dir, out_dir):
    
    list = glob(in_dir)
    for folder in list:
        patient_name = os.path.basename(os.path.normpath(folder))
        fileName = os.path.join(out_dir, patient_name + '.nii.gz')
        dicom2nifti.dicom_series_to_nifti(folder, fileName)


In [ ]:
# dicom2nifti.settings.disable_validate_slice_increment()

In [ ]:
in_path_img = '/content/drive/MyDrive/Dataset/decom_groups/images/*'
in_path_labels = '/content/drive/MyDrive/Dataset/decom_groups/labels/*'

out_path_img = '/content/drive/MyDrive/Dataset/Data_Train_Test/TrainVolumes'
out_path_labels = '/content/drive/MyDrive/Dataset/Data_Train_Test/TrainSegmentation'

# dicom_to_nifti(in_path_img,out_path_img) #for images
dicom_to_nifti(in_path_labels,out_path_labels) #for mask or segments.

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#Function to find if their an empty files in the dataset.


def empty_mask(in_dir):
        
    list_patients = []
    for patient in glob(os.path.join(in_dir, '*')):
        img = nib.load(patient)

        if len(np.unique(img.get_fdata())) ==1 :
            print(os.path.basename(os.path.normpath(patient)))
            list_patients.append(os.path.basename(os.path.normpath(patient)))
    
    return list_patients

In [ ]:
in_path = '/content/drive/MyDrive/Dataset/Data_Train_Test/TrainSegmentation/'
list_paitents = empty_mask(in_path)